In [6]:
import numpy as np 
import torch, time, copy
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torchvision import models
import os




In [7]:
custom_data_path = "./data/combined/"
max_path = "./data/average_vals.npy"


In [8]:
class customDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, mode, max_val_path, transform = None):
        """
        root_dir: Directory with all the np data
        transform: Optional transform
        """
        self.data_path = root_dir
        self.transform = transform
        self.mode = mode
        
        self.max_features = np.load(max_val_path)
        train_data = []
        test_data = []
        
        for filename in os.listdir(self.data_path):
            #print(filename)
            
            index = int(filename[:-4])
            
            if index % 10 == 0:
                test_data.append(np.load(root_dir + filename))
            else:
                train_data.append(np.load(root_dir + filename))
                
            
        self.data = train_data if mode == "train" else test_data
            
        
    
    def __len__ (self):
        return len(self.data)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        #print(np.array(self.data).shape)
        #print(self.data)
        real_data = self.data[idx][1:].astype(float)
        real_data = real_data / self.max_features
        label = self.data[idx][0]
        
        if self.transform:
            real_data = self.transform(real_data)
            
        return real_data, label
    
    
        
        

In [9]:
train_dataset = customDataset(custom_data_path, "train", max_val_path = max_path)
train_dataloader = DataLoader(train_dataset, batch_size = 16, shuffle = True, num_workers = 0)
test_dataset = customDataset(custom_data_path, "test", max_val_path = max_path,)
test_dataloader = DataLoader(test_dataset, batch_size = 16, num_workers = 0)

dataloaders = {"train": train_dataloader, "test": test_dataloader}
dataset_sizes = {"train": len(train_dataset), "test": len(test_dataset)}

print(dataset_sizes)

{'train': 1064, 'test': 118}


In [26]:
def train_model(Data_loader, model, loss_criterion, optimizer, dataset_sizes, num_epochs):
    
    best_acc = 0.0
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    
    for epoch in range(num_epochs):
        
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print ("-" * 10)
        
        
        for phase in ['train', 'test']:
            
            #Shouldn't we close model.train() for testing data?
            model.train()
        
            running_loss = 0.0
            running_corrects = 0
            
        
            #Forward Training
            for sample, label in Data_loader[phase]:
                #Clear gradients
                optimizer.zero_grad()

                sample = sample.float().squeeze()
                label = label.float().squeeze()
                
                with torch.set_grad_enabled(phase == 'train'):
                    output = model(sample).squeeze()
                    print("Type of output{} & type of label{}".format(type(output), type(label)))
                    print("This is output{}".format(type(output)))
                    preds = torch.round(output)
                    loss = loss_criterion(output, label)
                    #print("This is preds{}".format(preds))
                    #Backpropogate the loss and update the parameters accordingly
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                #statistics
                running_loss += loss.item()
                #print(preds.size())
                #Need to use torch.sum because torch.eq(a,b) still returns a torch
                running_corrects += torch.sum(preds == label)
            
            epoch_loss = running_loss/dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{}Loss: {:.2f} Acc: {:.3f}'.format(phase,epoch_loss, epoch_acc))
            
            #deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            
            #Are we comparing the model every 10 data samples or every epoch?
            print('Finished training')

        
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}'.format(time_elapsed//60, time_elapsed % 60))
            #Print Statistics
    print('Best val Acc:{:4f}'.format(best_acc))
    
    model.load_state_dict(best_model_wts)
    
    return model 
        #Why don't I need to do model.eval()?
        #model.eval()
        
    '''for sample, label in Data_loader["test"]:
            pred = model(sample)
            val_loss = loss_criterion(pred,label)
    '''
        

In [27]:
model = nn.Sequential(nn.Linear(50,25), nn.Sigmoid(),nn.Linear(25,10),nn.Linear(10,1), nn.Sigmoid())
#criterion = nn.MSELoss()
criterion = nn.BCELoss() #we can try hinge loss or Euclidiean distance

#optimizer = optim.SGD(model.parameters(), lr = 0.05, momentum = 0.9)
optimizer = optim.Adam(model.parameters(),lr=0.03, weight_decay=0.00001)
#small weight decays work better seem to suggest that the weights are relatively small

#Works fairly well around 91%



In [28]:
'''model = nn.Sequential(nn.Linear(50,25), nn.Linear(25,1), nn.Sigmoid())
#criterion = nn.MSELoss()
criterion = nn.BCELoss() #we can try hinge loss or Euclidiean distance

#optimizer = optim.SGD(model.parameters(), lr = 0.05, momentum = 0.9)
optimizer = optim.Adam(model.parameters(),lr=0.05, weight_decay=0.001)
#small weight decays work better seem to suggest that the weights are relatively small
'''

'model = nn.Sequential(nn.Linear(50,25), nn.Linear(25,1), nn.Sigmoid())\n#criterion = nn.MSELoss()\ncriterion = nn.BCELoss() #we can try hinge loss or Euclidiean distance\n\n#optimizer = optim.SGD(model.parameters(), lr = 0.05, momentum = 0.9)\noptimizer = optim.Adam(model.parameters(),lr=0.05, weight_decay=0.001)\n#small weight decays work better seem to suggest that the weights are relatively small\n'

In [29]:
'''model = nn.Sequential(nn.Linear(50,25, bias = False), nn.Sigmoid(),nn.Linear(25,10, bias = False),nn.ReLU(), nn.Linear(10,1, bias = False), nn.Sigmoid())
#criterion = nn.MSELoss()
criterion = nn.BCELoss() #we can try hinge loss or Euclidiean distance

#optimizer = optim.SGD(model.parameters(), lr = 0.05, momentum = 0.9)
optimizer = optim.Adam(model.parameters(),lr=0.05, weight_decay=0.0001)
#small weight decays work better seem to suggest that the weights are relatively small
'''

'model = nn.Sequential(nn.Linear(50,25, bias = False), nn.Sigmoid(),nn.Linear(25,10, bias = False),nn.ReLU(), nn.Linear(10,1, bias = False), nn.Sigmoid())\n#criterion = nn.MSELoss()\ncriterion = nn.BCELoss() #we can try hinge loss or Euclidiean distance\n\n#optimizer = optim.SGD(model.parameters(), lr = 0.05, momentum = 0.9)\noptimizer = optim.Adam(model.parameters(),lr=0.05, weight_decay=0.0001)\n#small weight decays work better seem to suggest that the weights are relatively small\n'

In [30]:
final_model = train_model(dataloaders, model, criterion, optimizer, dataset_sizes, num_epochs = 200)

Epoch 0/199
----------
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This i

Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.T

Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.T

Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
trainLoss: 0.02 Acc: 0.840
Finished training
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 

Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.T

...

Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.T

Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.T

Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.T

Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.T

Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.Tensor'>
Type of output<class 'torch.Tensor'> & type of label<class 'torch.Tensor'>
This is output<class 'torch.T

In [402]:
print(final_model)

Sequential(
  (0): Linear(in_features=50, out_features=25, bias=False)
  (1): Sigmoid()
  (2): Linear(in_features=25, out_features=10, bias=False)
  (3): Linear(in_features=10, out_features=1, bias=False)
  (4): Sigmoid()
)


In [403]:
#Why are there this many parameters
for param in final_model.parameters():
    print(param)
    print('-'*10)

Parameter containing:
tensor([[ 0.0982, -0.9125, -0.1178,  ...,  2.5939,  0.3221, -0.3829],
        [-0.5498, -0.8320, -0.5584,  ..., -0.7524, -0.8987, -0.5155],
        [ 0.5739,  0.6641,  0.5472,  ...,  1.3210,  1.5166,  1.0743],
        ...,
        [-0.4179, -0.6509, -0.2395,  ...,  1.0087, -0.4911, -0.8211],
        [ 0.7918,  0.7581,  0.7911,  ...,  0.7875,  0.8814,  0.9996],
        [-0.2687,  1.4013, -0.5765,  ..., -0.6395,  0.2008,  0.3169]],
       requires_grad=True)
----------
Parameter containing:
tensor([[ 9.6768e-02, -6.8419e-02,  8.3504e-01, -1.0061e-02,  1.1624e-01,
          1.2227e+00, -3.5225e-01,  4.0365e-02, -1.3461e-01,  6.6104e-02,
         -1.1305e-02, -3.1385e-02,  3.3935e-01,  2.0805e-03,  2.9270e-03,
          4.3836e-03,  1.3805e-01, -5.5828e-01, -5.9980e-03,  8.4914e-02,
          1.7889e-01, -4.6961e-01, -2.6349e-01,  2.0347e-01,  2.9943e-01],
        [-3.3124e-02, -6.5140e-02,  3.6342e-01,  3.3944e-02,  9.5395e-02,
         -2.2859e-01, -9.2940e-01, -2.1

In [410]:
'''
1. What was the rationale for not adding bias?
2. Did we automatically do the testing accuracy while training the model?
Why didn't we need to use model.eval() or turn off model.train() when label == 'train'?
3. Still don't really understand how batching & epoch works. 
4. I don't understand how Conv1D would help. Are they not just for images?
'''

'\n1. What was the rationale for not adding bias?\n2. Did we automatically do the testing accuracy while training the model?\nwhy\n'